In [1]:
!python -m spacy download fr_core_news_sm

2021-05-28 13:35:26.157305: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-05-28 13:35:26.157337: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


[+] Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


In [2]:
#Importing required packages
import pandas as pd
import seaborn as sns
import nltk
import matplotlib.pyplot as plt
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import numpy as np
!python -m pip install -U gensim
!python -m pip install -U keras
!python -m pip install tensorflow
from sklearn import datasets
from sklearn.base import TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.utils.multiclass import unique_labels
import string
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
import re
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import numpy as np

Requirement already up-to-date: gensim in c:\users\adria\anaconda3\lib\site-packages (4.0.1)
Requirement already up-to-date: keras in c:\users\adria\anaconda3\lib\site-packages (2.4.3)


In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adria\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
#Import data and remove punctuation
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text
Sentences = "https://raw.githubusercontent.com/TetraFaal/Big-Scale-Analytics-2021-Tesla/main/Data/Dataset_Aicrowd/22fdbaea-415d-4685-8132-1916959ca359_train.csv"
Sentences = pd.read_csv(Sentences)
Sentences.columns = ['Text','Difficulty']

Sentences["Text"] = Sentences['Text'].apply(remove_punctuations)

Cognates = "https://raw.githubusercontent.com/TetraFaal/Big-Scale-Analytics-2021-Tesla/main/Data/Cognates/Cognates.csv"
Cognates = pd.read_csv(Cognates)

CognatesGlossary = "https://raw.githubusercontent.com/TetraFaal/Big-Scale-Analytics-2021-Tesla/main/Data/Cognates/glossaryCognates.csv"
CognatesGlossary = pd.read_csv(CognatesGlossary)

In [5]:
Sentence2 = Sentences
[(lambda x: re.sub(r'|'.join(Cognates), '', x))(x) for x in Sentence2]
Sentence2.head(-1)

,Text,Difficulty
0,Cest pour quand,A1
1,Je pense que cest bon,A1
2,Cest pas mal,A1
3,Questce que vous faites,A1
4,Cest bien,A1
...,...,...
4794,Le jardin commença à fleurir après quil se fut...,C2
4795,Le problème va du garçon de café tentant inint...,C2
4796,Les lois de programmation mentionnées à lantép...,C2
4797,La conscience malheureuse cest donc le mauvais...,C2


In [6]:
Cognates.head()

,Cognates
0,abattoir
1,abdication
2,abdominal
3,aberrant
4,aberration


In [7]:
CognatesGlossary.head()

,CognatesEN,CognatesFR
0,absolutely,absolument
1,abundance,abondance
2,abuse,abuser
3,accelerate (to),accélérer
4,accentuate (to),accentuer


In [8]:
#Create a cognates list and delete duplicates
ListCo = []
for cognate  in CognatesGlossary.CognatesFR:
  ListCo.append(cognate)
for cognate  in Cognates.Cognates:
  ListCo.append(cognate)
ListCognates = list(set(ListCo))
print(ListCognates)
print(len(ListCognates))

['circulaire', 'adverbial', 'apparition', 'olive', 'structural', 'escalator', 'machination', 'purge', 'naturellement', 'initiation', 'diminution', 'parasite', 'uniforme', 'cabine', 'division', 'international', 'piccolo', 'repentant', 'reproduction', 'proportion', 'bourgeois', 'suicide', 'nécessité', 'opus', 'sorbet', 'canal', 'imagination', 'collage', 'institution', 'propulsion', 'torture', 'accord', 'quiet', 'insoluble', 'représenter', 'agence', 'instrument', 'venimeux', 'placenta', 'prude', 'fameux', 'compliment', 'identifiable', 'orchestral', 'vigilance', 'tandem', 'bicyclette', 'africain', 'contusion', 'usuelle', 'normalement', 'diagonal', 'submersion', 'scalp', 'national', 'observation', 'duplex', 'budget', 'omniscient', 'civiliser', 'bikini', 'baptismal', 'tomate', 'confinement', 'irritant', 'neutron', 'sanatorium', 'social', 'bigot', 'juvénile', 'polka', 'menstruation', 'combat', 'suppression', 'sardine', 'rotor', 'exploitation', 'indigestion', 'moral', 'incrédible', 'horizon', 

In [9]:
pd.set_option("max_colwidth", None)
Sentences.head(10)


,Text,Difficulty
0,Cest pour quand,A1
1,Je pense que cest bon,A1
2,Cest pas mal,A1
3,Questce que vous faites,A1
4,Cest bien,A1
5,Ce nest pas juste,A1
6,Je ne veux pas,A1
7,Je ne sais pas,A1
8,Plus que ça,A1
9,Non je ne le trouve pas,A1


In [10]:
#Calculating base rate (perfect base rate with 6 features is 0.16)
max(len(Sentence2[Sentence2["Difficulty"] =='A1']) / len(Sentence2), len(Sentence2[Sentence2["Difficulty"] == 'A2']) / len(Sentence2), len(Sentence2[Sentence2["Difficulty"] == 'B1']) / len(Sentence2) , len(Sentence2[Sentence2["Difficulty"] == 'B2']) / len(Sentence2),len(Sentence2[Sentence2["Difficulty"] == 'C1']) / len(Sentence2), len(Sentence2[Sentence2["Difficulty"] == 'C2']) / len(Sentence2))

0.16666666666666666

In [11]:
#Counting targets values 
display(Sentence2.Difficulty.value_counts())

B2    800
A2    800
B1    800
A1    800
C1    800
C2    800
Name: Difficulty, dtype: int64

In [12]:
indexMax = (Sentence2.shape[0] -1)
Sentence2.shape[0]

4800

In [13]:
sp = spacy.load('fr_core_news_sm')

In [14]:
data = list(zip(Sentence2["Text"],Sentence2["Difficulty"]))
datawords = []
for t, label in data:
  list_of_strings = sp(t)
  token = [i.text for i in list_of_strings]
  datawords.append([token, label])
print(datawords)

[[['Cest', 'pour', 'quand'], 'A1'], [['Je', 'pense', 'que', 'cest', 'bon'], 'A1'], [['Cest', 'pas', 'mal'], 'A1'], [['Questce', 'que', 'vous', 'faites'], 'A1'], [['Cest', 'bien'], 'A1'], [['Ce', 'nest', 'pas', 'juste'], 'A1'], [['Je', 'ne', 'veux', 'pas'], 'A1'], [['Je', 'ne', 'sais', 'pas'], 'A1'], [['Plus', 'que', 'ça'], 'A1'], [['Non', 'je', 'ne', 'le', 'trouve', 'pas'], 'A1'], [['Questce', 'que', 'ça', 'veut', 'dire'], 'A1'], [['Cest', 'déjà', 'ça'], 'A1'], [['Je', 'men', 'vais'], 'A1'], [['Je', 'ne', 'comprends', 'pas'], 'A1'], [['Je', 'ne', 'te', 'crois', 'pas'], 'A1'], [['Je', 'ne', 'me', 'sens', 'pas', 'bien'], 'A1'], [['Tu', 'ne', 'comprends', 'pas'], 'A1'], [['Cest', 'pour', 'la', 'photo'], 'A1'], [['Je', 'suis', 'désolée', 'elle', 'nest', 'pas', 'là'], 'A1'], [['Je', 'nai', 'pas', 'bien', 'compris'], 'A1'], [['Je', 'suis', 'daccord'], 'A1'], [['Ce', 'nest', 'pas', 'important'], 'A1'], [['Et', 'toi'], 'A1'], [['Mais', 'il', 'y', 'a', 'un', 'problème'], 'A1'], [['Estce', 'que'

In [15]:
newdf = pd.DataFrame(datawords, columns=["Text", "Label"])

In [16]:
newdf.head(20)

,Text,Label
0,"[Cest, pour, quand]",A1
1,"[Je, pense, que, cest, bon]",A1
2,"[Cest, pas, mal]",A1
3,"[Questce, que, vous, faites]",A1
4,"[Cest, bien]",A1
5,"[Ce, nest, pas, juste]",A1
6,"[Je, ne, veux, pas]",A1
7,"[Je, ne, sais, pas]",A1
8,"[Plus, que, ça]",A1
9,"[Non, je, ne, le, trouve, pas]",A1


In [17]:
#create NumberOfCognate and Cognates columns
newdf = newdf.assign(NumberOfCognate = 0) 
newdf = newdf.assign(Cognates = "")
newdf.head() 

,Text,Label,NumberOfCognate,Cognates
0,"[Cest, pour, quand]",A1,0,
1,"[Je, pense, que, cest, bon]",A1,0,
2,"[Cest, pas, mal]",A1,0,
3,"[Questce, que, vous, faites]",A1,0,
4,"[Cest, bien]",A1,0,


In [18]:
index = 0
for sentence in newdf.Text:
  score=0
  liste =[]
  for word in sentence:
    if str(word) in ListCognates:
      score+=1
      liste.append(word)
  if score == 0 and index <= indexMax :
    liste = ''
  if index <=indexMax:
    newdf.at[index,'NumberOfCognate']= int(score)
    newdf.at[index,'Cognates']= np.array(liste)
    index += 1


In [19]:
newdf.head()

,Text,Label,NumberOfCognate,Cognates
0,"[Cest, pour, quand]",A1,0,
1,"[Je, pense, que, cest, bon]",A1,0,
2,"[Cest, pas, mal]",A1,0,
3,"[Questce, que, vous, faites]",A1,0,
4,"[Cest, bien]",A1,0,


In [20]:
cognatesSum = newdf.NumberOfCognate.sum()
print("We found in total : ", cognatesSum, "cognates.")
print("The sentence with the most cognates is :")
newdf.sort_values(by=['NumberOfCognate'],ascending=False).head(1)

We found in total :  3493 cognates.
The sentence with the most cognates is :


,Text,Label,NumberOfCognate,Cognates
4787,"[Paul, Andreu, larchitecte, de, laéroport, de, Roissy, du, terminal, 1, au, terminal, 2EK, ainsi, que, de, lOpéra, de, Pékin, entre, autres, sétait, demandé, sil, devait, installer, des, portes, automatiques, dans, un, aéroport, quil, était, en, train, de, concevoir, comme, le, requérait, tout, aéroport, digne, de, ce, nom, car, après, un, rapide, calcul, à, partir, des, conditions, locales, le, coût, dachat, et, surtout, de, maintenance, dune, porte, automatique, revenait, plus, cher, que, le, salaire, dun, portier, qui, pouvait, nourrir, une, famille, avec, ce, que, la, porte, était, bien, incapable, de, faire]",C2,9,"[terminal, terminal, maintenance, dune, automatique, plus, salaire, famille, incapable]"


In [21]:
del newdf['NumberOfCognate']

newdf

,Text,Label,Cognates
0,"[Cest, pour, quand]",A1,
1,"[Je, pense, que, cest, bon]",A1,
2,"[Cest, pas, mal]",A1,
3,"[Questce, que, vous, faites]",A1,
4,"[Cest, bien]",A1,
...,...,...,...
4795,"[Le, problème, va, du, garçon, de, café, tentant, inintelligiblement, de, communiquer, avec, des, touristes, médusés, jusque, comme, on, la, vu, aux, niveaux, les, plus, élevés, de, la, société, française]",C2,"[problème, plus]"
4796,"[Les, lois, de, programmation, mentionnées, à, lantépénultième, alinéa, de, larticle, 34, de, la, Constitution, ont, pour, objet, de, déterminer, les, objectifs, de, laction, de, lÉtat]",C2,"[objet, déterminer]"
4797,"[La, conscience, malheureuse, cest, donc, le, mauvais, infini, tel, que, la, philosophie, de, Fichte, amène, à, le, concevoir]",C2,[conscience]
4798,"[Comme, Monsieur, un, informaticien, fondateur, de, la, librairie, des, maths, pour, qui, il, y, a, tellement, de, manières, daimer, les, mathématiques, quil, propose, de, nombreux, produits, plus, ou, moins, littéraires, plus, ou, moins, ludiques]",C2,"[plus, plus]"


In [22]:
newdf = newdf[['Text', 'Cognates', 'Label']]
newdf

,Text,Cognates,Label
0,"[Cest, pour, quand]",,A1
1,"[Je, pense, que, cest, bon]",,A1
2,"[Cest, pas, mal]",,A1
3,"[Questce, que, vous, faites]",,A1
4,"[Cest, bien]",,A1
...,...,...,...
4795,"[Le, problème, va, du, garçon, de, café, tentant, inintelligiblement, de, communiquer, avec, des, touristes, médusés, jusque, comme, on, la, vu, aux, niveaux, les, plus, élevés, de, la, société, française]","[problème, plus]",C2
4796,"[Les, lois, de, programmation, mentionnées, à, lantépénultième, alinéa, de, larticle, 34, de, la, Constitution, ont, pour, objet, de, déterminer, les, objectifs, de, laction, de, lÉtat]","[objet, déterminer]",C2
4797,"[La, conscience, malheureuse, cest, donc, le, mauvais, infini, tel, que, la, philosophie, de, Fichte, amène, à, le, concevoir]",[conscience],C2
4798,"[Comme, Monsieur, un, informaticien, fondateur, de, la, librairie, des, maths, pour, qui, il, y, a, tellement, de, manières, daimer, les, mathématiques, quil, propose, de, nombreux, produits, plus, ou, moins, littéraires, plus, ou, moins, ludiques]","[plus, plus]",C2


In [23]:
df=pd.DataFrame()
df['Text'] = newdf['Text'].astype('<U32') + newdf['Cognates'].astype('<U32')
df['label'] = newdf['Label']
df

,Text,label
0,"['Cest', 'pour', 'quand']",A1
1,"['Je', 'pense', 'que', 'cest', 'bon']",A1
2,"['Cest', 'pas', 'mal']",A1
3,"['Questce', 'que', 'vous', 'faites']",A1
4,"['Cest', 'bien']",A1
...,...,...
4795,"['Le', 'problème', 'va', 'du', 'garçon', 'de', 'café', 'tentant', 'inintelligiblement', 'de', 'communiquer', 'avec', 'des', 'touristes', 'médusés', 'jusque', 'comme', 'on', 'la', 'vu', 'aux', 'niveaux', 'les', 'plus', 'élevés', 'de', 'la', 'société', 'française']['problème' 'plus']",C2
4796,"['Les', 'lois', 'de', 'programmation', 'mentionnées', 'à', 'lantépénultième', 'alinéa', 'de', 'larticle', '34', 'de', 'la', 'Constitution', 'ont', 'pour', 'objet', 'de', 'déterminer', 'les', 'objectifs', 'de', 'laction', 'de', 'lÉtat']['objet' 'déterminer']",C2
4797,"['La', 'conscience', 'malheureuse', 'cest', 'donc', 'le', 'mauvais', 'infini', 'tel', 'que', 'la', 'philosophie', 'de', 'Fichte', 'amène', 'à', 'le', 'concevoir']['conscience']",C2
4798,"['Comme', 'Monsieur', 'un', 'informaticien', 'fondateur', 'de', 'la', 'librairie', 'des', 'maths', 'pour', 'qui', 'il', 'y', 'a', 'tellement', 'de', 'manières', 'daimer', 'les', 'mathématiques', 'quil', 'propose', 'de', 'nombreux', 'produits', 'plus', 'ou', 'moins', 'littéraires', 'plus', 'ou', 'moins', 'ludiques']['plus' 'plus']",C2


In [24]:
# Iformation about our dataset
display(df.shape, df.info(), df.describe)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4800 entries, 0 to 4799
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    4800 non-null   object
 1   label   4800 non-null   object
dtypes: object(2)
memory usage: 75.1+ KB


(4800, 2)

None

<bound method NDFrame.describe of                                                                                                                                                                                                                                                                                                                                              Text  \
0                                                                                                                                                                                                                                                                                                                       ['Cest', 'pour', 'quand']   
1                                                                                                                                                                                                                                                                                           

In [25]:
#Preprocessed data for Google CLoud AutoML
#df.to_csv('Data_Tokenizer_cognates_whitout_punctuation.csv', encoding='utf-8-sig', index=False)

PART 2 - Building the camemBERT Model

In [26]:
!pip install transformers

In [30]:
!pip install torchvision 

In [31]:
import torch
from tqdm import tqdm, trange


# Importing specific libraries for data prerpcessing, model archtecture choice, training and evaluation
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import CamembertTokenizer, CamembertForSequenceClassification
from transformers import AdamW
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

In [32]:
epochs = 5
MAX_LEN = 128
batch_size = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [33]:
pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [34]:
tokenizer = CamembertTokenizer.from_pretrained('camembert-base',do_lower_case=True)

In [35]:
# Creates list of texts and labels
# Replace labels by numbers to make the model work

Sentence2['Difficulty'] = Sentence2['Difficulty'].replace(['A1','A2','B1',"B2", "C1", "C2"],[0,1,2,3,4,5]) 
text = Sentence2['Text'].to_list()
labels = Sentence2['Difficulty'].to_list()

#user tokenizer to convert sentences into tokenizer
input_ids  = [tokenizer.encode(sent,add_special_tokens=True,max_length=MAX_LEN) for sent in text]

# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]  
    attention_masks.append(seq_mask)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [36]:
Sentence2["Difficulty"].head(-1)

0       0
1       0
2       0
3       0
4       0
       ..
4794    5
4795    5
4796    5
4797    5
4798    5
Name: Difficulty, Length: 4799, dtype: int64

In [37]:
# Use train_test_split to split our data into train and validation sets for training
train_inputs, validation_inputs, train_labels, validation_labels, train_masks, validation_masks = train_test_split(input_ids, labels, attention_masks,
                                                            random_state=42, test_size=0.1)

# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [38]:
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 
model = CamembertForSequenceClassification.from_pretrained("camembert-base", num_labels=6)
model.to(device)

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 

CamembertForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32005, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (Laye

In [39]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, eps=10e-8)

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [40]:
# Store our loss and accuracy for plotting if we want to visualize training evolution per epochs after the training process
train_loss_set = []

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):  
    # Tracking variables for training
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
  
    # Train the model
    model.train()
    for step, batch in enumerate(train_dataloader):
        # Add batch to device CPU or GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()
        # Forward pass
        outputs = model(b_input_ids,token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
  # Get loss value
        loss = outputs[0]
        # Add it to train loss list
        train_loss_set.append(loss.item())    
        # Backward pass
        loss.backward()
        # Update parameters and take a step using the computed gradient
        optimizer.step()
    
        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    


    # Tracking variables for validation
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    # Validation of the model
    model.eval()
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        # Add batch to device CPU or GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Telling the model not to compute or store gradients, saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions
            outputs =  model(b_input_ids,token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            loss, logits = outputs[:2]
    
        # Move logits and labels to CPU if GPU is used
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Train loss: 1.482560462752978


Epoch:  20%|██        | 1/5 [26:02<1:44:11, 1562.85s/it]

Validation Accuracy: 0.484375
Train loss: 1.0760775439441204


Epoch:  40%|████      | 2/5 [50:21<1:16:34, 1531.56s/it]

...

Validation Accuracy: 0.5166666666666667
Train loss: 0.7209270514547825


Epoch:  80%|████████  | 4/5 [1:32:12<23:11, 1391.84s/it]

Validation Accuracy: 0.5166666666666667
Train loss: 0.5527455457796653


Epoch: 100%|██████████| 5/5 [1:53:05<00:00, 1357.08s/it]

Validation Accuracy: 0.5510416666666667


In [57]:
from sklearn import model_selection
import pickle


In [42]:
comments = ["coucou ca va"]

In [81]:
# Encode the comments
tokenized_comments_ids = [tokenizer.encode(comment,add_special_tokens=True,max_length=MAX_LEN) for comment in comments]
# Pad the resulted encoded comments
tokenized_comments_ids = pad_sequences(tokenized_comments_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Create attention masks 
attention_masks = []
for seq in tokenized_comments_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

prediction_inputs = torch.tensor(tokenized_comments_ids)
prediction_masks = torch.tensor(attention_masks)

In [82]:
# Apply the finetuned model (Camembert)
flat_pred = []
with torch.no_grad():
    # Forward pass, calculate logit predictions
    outputs =  model(prediction_inputs.to(device),token_type_ids=None, attention_mask=prediction_masks.to(device))
    logits = outputs[0]
    logits = logits.detach().cpu().numpy() 
    flat_pred.extend(np.argmax(logits, axis=1).flatten())

In [45]:
for i in range(len(flat_pred)):
    print('Comment: ', comments[i])
    print('Label', flat_pred[i])

Comment:  coucou ca va
Label 0


In [46]:
# save the model to disk
filename = 'withoutcognates_model.sav'
pickle.dump(model, open(filename, 'wb'))

Look at the second NoteBook for the use of this model and the prediction for AICrowd Dataset